In [1]:
import torch
import torch.nn as nn 
from torchvision import transforms
import torchvision.datasets as dsets
from torch.utils.data import DataLoader 
from tqdm.auto import tqdm
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
torch.manual_seed(0)

In [2]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [3]:
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True))

In [4]:
class Generator(nn.Module):
    def __init__(self, z_dim = 10, im_dim = 784, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim*2),
            get_generator_block(hidden_dim*2, hidden_dim*4),
            get_generator_block(hidden_dim*4, hidden_dim*8),
            nn.Linear(hidden_dim*8, im_dim), 
            nn.Sigmoid())
        
    def forward(self, noise):   
        return self.gen(noise)
    
    def get_gen(self):
        
        return self.gen
    

In [5]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples, z_dim)

In [6]:
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
          (https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html)
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2))

In [7]:
class Discriminator(nn.Module):
    def __init__(self, im_dim = 784, hidden_dim = 128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim*4),
            get_discriminator_block(hidden_dim*4, hidden_dim*2),
            get_discriminator_block(hidden_dim*2, hidden_dim),
            nn.Linear(hidden_dim, 1))
        
    def forward(self, image):
        return self.disc(image)
            
    def get_disc(self):
        return self.disc

In [8]:
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001

train_dataset = dsets.FashionMNIST(root = './data', download = True, transform = transforms.ToTensor())

dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

device = 'cpu'

In [9]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr = lr)

disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(disc.parameters(), lr = lr)

In [10]:
'''


  Lossfake =−(target×log(prediction)+(1−target)×log(1−prediction))

Fake predictions: [0.2, 0.8, 0.6, 0.3, 0.9]
Target label: 0 (indicating fake)

Using binary cross-entropy loss, the formula for each element of the loss is:
Lossfake=−(target×log⁡(prediction)+(1−target)×log⁡(1−prediction))
Lossfake​=−(target×log(prediction)+(1−target)×log(1−prediction))

Let's calculate the loss for each fake image:

    For the first prediction (0.2):
    Loss1= −(0×log⁡(0.2)+(1−0)×log⁡(1−0.2)) ≈ 0.223

    For the second prediction (0.8):
    Loss2= −(0×log⁡(0.8)+(1−0)×log⁡(1−0.8)) ≈ 0.223


    For the third prediction (0.6):
    Loss3= −(0×log⁡(0.6)+(1−0)×log⁡(1−0.6)) ≈ 0.511

    For the fourth prediction (0.3):
    Loss4= −(0×log⁡(0.3)+(1−0)×log⁡(1−0.3)) ≈ 0.357

    For the fifth prediction (0.9):
    Loss5= −(0×log⁡(0.9)+(1−0)×log⁡(1−0.9)) ≈ 0.105

Now, we'll take the average of these losses to get the overall fake loss.
         Average Loss ≈ 0.2838

'''

"\n\n\n  Lossfake =−(target×log(prediction)+(1−target)×log(1−prediction))\n\nFake predictions: [0.2, 0.8, 0.6, 0.3, 0.9]\nTarget label: 0 (indicating fake)\n\nUsing binary cross-entropy loss, the formula for each element of the loss is:\nLossfake=−(target×log\u2061(prediction)+(1−target)×log\u2061(1−prediction))\nLossfake\u200b=−(target×log(prediction)+(1−target)×log(1−prediction))\n\nLet's calculate the loss for each fake image:\n\n    For the first prediction (0.2):\n    Loss1= −(0×log\u2061(0.2)+(1−0)×log\u2061(1−0.2)) ≈ 0.223\n\n    For the second prediction (0.8):\n    Loss2= −(0×log\u2061(0.8)+(1−0)×log\u2061(1−0.8)) ≈ 0.223\n\n\n    For the third prediction (0.6):\n    Loss3= −(0×log\u2061(0.6)+(1−0)×log\u2061(1−0.6)) ≈ 0.511\n\n    For the fourth prediction (0.3):\n    Loss4= −(0×log\u2061(0.3)+(1−0)×log\u2061(1−0.3)) ≈ 0.357\n\n    For the fifth prediction (0.9):\n    Loss5= −(0×log\u2061(0.9)+(1−0)×log\u2061(1−0.9)) ≈ 0.105\n\nNow, we'll take the average of these losses to ge

In [44]:
fake_noise = get_noise(5, z_dim, device)
fake = gen(fake_noise).detach()
print("gen img shape: ", fake.shape)
disc_fake_pred = disc(fake)
print("disc tensor shape: ", disc_fake_pred.shape)
print(disc_fake_pred)
disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
print(disc_fake_loss)

gen img shape:  torch.Size([5, 784])
disc tensor shape:  torch.Size([5, 1])
tensor([[-0.4969],
        [-0.3424],
        [-0.1927],
        [-0.4492],
        [-0.7467]], grad_fn=<AddmmBackward0>)
tensor(0.4989, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [20]:
def get_disc_loss(gen, disc, criterion, real, loss, num_images, z_dim, device):
    
    fake_noise = get_noise(num_images, z_dim, device)
    fake = gen(fake_noise).detach()
    
    disc_fake_pred = disc(fake)
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
                               
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    
    disc_loss = (disc_fake_loss + disc_real_loss)/2.0
    
    return disc_loss

In [21]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    
    fake_noise = get_noise(num_images, z_dim)
    
    fake_images = gen(fake_noise)
    
    disc_fake_pred = disc(fake_images)
    
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    
    return gen_loss

In [ ]:
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True # Whether the generator should be tested
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, criterion, cur_batch_size, z_dim, device=device)

        # Update gradients
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        ### Update generator ###
        # Zero out the gradients.
        gen_opt.zero_grad()
        # Calculate the generator loss
        #fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        #fake = gen(fake_noise)
        #disc_fake_pred = disc(fake)
        #gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        # Backprop through the generator: update the gradients and optimizer.
        gen_loss.backward()
        gen_opt.step()

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1
